### Import Libraries

In [3]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

tfds.disable_progress_bar()

2023-11-07 11:05:24.678663: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 11:05:24.929172: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 11:05:24.931348: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 11:05:26.054583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def plot_graphs(history, metric):
	plt.plot(history.history[metric])
	plt.plot(history.history['val_'+metric], '')
	plt.xlabel("Epochs")
	plt.ylabel(metric)
	plt.legend([metric, 'val_'+metric])

### Load Dataset

In [5]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset.element_spec

2023-11-07 11:06:29.784312: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dataset imdb_reviews downloaded and prepared to /home/ubuntu/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [6]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


2023-11-07 11:13:17.601427: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b'I loathed this film. The original Phantasm had such wonderful ambiance and mystery. Like many 70s horror flicks, it looked and felt like some creepy, unfinished documentary. Phantasm II, from the late 80s, pumped up the action, but maintained this nice attention to mood. Sadly, Phantasm III is just awful. It tediously explains all of the weird happenings in the previous films, which diminishes rather than expands their power. It shamelessly degrades imagery from the first Phantasm like a cheap reenactment of the original. There are so many flying spheres in this movie that they seem more like household pests than menacing death orbs. Hundreds hang from the ceiling like Christmas balls swaying in the draft. Didn\'t anyone-- the prop master, the DP, the editor, the director-- notice or care that they looked so crummy? Even worse, Phantasm III presents one corny, unfunny joke after another. How different from the intensity of the first film. The original Phantasm used humor to 

2023-11-07 11:13:29.823482: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [8]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [12]:
vocab=np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [13]:
encoded_example=encoder(example)[:3].numpy()
encoded_example

array([[ 10,   1,  11, ...,   0,   0,   0],
       [  9,   7,  22, ...,   0,   0,   0],
       [ 10, 364,  51, ...,   0,   0,   0]])

In [14]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'I loathed this film. The original Phantasm had such wonderful ambiance and mystery. Like many 70s horror flicks, it looked and felt like some creepy, unfinished documentary. Phantasm II, from the late 80s, pumped up the action, but maintained this nice attention to mood. Sadly, Phantasm III is just awful. It tediously explains all of the weird happenings in the previous films, which diminishes rather than expands their power. It shamelessly degrades imagery from the first Phantasm like a cheap reenactment of the original. There are so many flying spheres in this movie that they seem more like household pests than menacing death orbs. Hundreds hang from the ceiling like Christmas balls swaying in the draft. Didn\'t anyone-- the prop master, the DP, the editor, the director-- notice or care that they looked so crummy? Even worse, Phantasm III presents one corny, unfunny joke after another. How different from the intensity of the first film. The original Phantasm used humor t

In [15]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [17]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [18]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

1/1 [==============================] - 2s 2s/step
[-0.00959009]


In [19]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

1/1 [==============================] - 0s 168ms/step
[-0.00959009]


In [20]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [22]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

Epoch 1/10


  2/391 [..............................] - ETA: 4:16 - loss: 0.6929 - accuracy: 0.5000 

2023-11-07 11:21:17.501219: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16416768 exceeds 10% of free system memory.
2023-11-07 11:21:17.505225: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16416768 exceeds 10% of free system memory.
2023-11-07 11:21:17.505520: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16416768 exceeds 10% of free system memory.
2023-11-07 11:21:17.516184: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16416768 exceeds 10% of free system memory.


  3/391 [..............................] - ETA: 4:14 - loss: 0.6933 - accuracy: 0.5312

2023-11-07 11:21:18.088344: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16416768 exceeds 10% of free system memory.


...

2023-11-07 11:28:57.753611: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


391/391 [==============================] - 227s 581ms/step - loss: 0.3780 - accuracy: 0.8267 - val_loss: 0.3669 - val_accuracy: 0.8417
Epoch 4/10


2023-11-07 11:32:45.154578: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


273/391 [===================>..........] - ETA: 1:08 - loss: 0.3493 - accuracy: 0.8502

KeyboardInterrupt: 